## Algorithm:

Step 1: Import os, request module, sqlite3 for database connection and beautiful soup library for pulling data.

Step 2: Check if commodity_prices.txt file already exists in your system or not, if found remove inside contents and open it or if not found create a file in that name.

Step 3: Check if there is any existing database under your given db name, if yes remove it and if not found create it. Use try-except to create that database under your given name.

Step 4: Create a object with Cursor class so that it can invoke methods which execute SQLite statements.

Step 5: Create table with the column name Ticker and Price, Ticker datatype is TEXT and Price datatype is REAL.

Step 6: Now call the website's url using requests library and save in a variable.

Step 7: Define a function that will parse the commodity price from saved website's data, commodity name will be sent as parameter.

Step 8: Make a list of all commodities which will be searched. Now we have to fetch price, write it in a file and use it in that created db.

Step 9: Use a for loop to iterate through items from previously created list (step 8). then write the Commodity Name and Commodity Price in the file; get the commodity price using the price_of_commodities function (defined in step 7). Insert the commodity name in as Ticker and commodity price as Price; by calling price_of_commodities function.

Step 10: Commit all changes in db, close cursor and close db connection.

Step 11: Now check by querying if your created db is giving write output or not. Import pandas.

Step 12: Close your connection to db after checking as in next run it will give error if keep opened.

In [7]:
import requests
import sqlite3
import bs4
import os

################### File Creation ###################
if os.path.exists('commodity_prices.txt'):                 # checking if file exists with the name "commodity_prices.txt"
    os.remove('commodity_prices.txt')                      # if found remove the file
    with open('commodity_prices.txt','w+') as fp:          # open the file with the name "commodity_prices.txt"
        pass
else:                                                      
    with open('commodity_prices.txt','w+') as fp:          # if file doesn't exist, open the file with the name "commodity_prices.txt"
        pass

    
################### DataBase and Table Creation ###################
conn = None
if os.path.exists('CommodityDatabase.db'):                 # checking if any DataBase file exists with the name "CommodityDatabase.db"
    os.remove('CommodityDatabase.db')                      # if found remove the DB file
    try:
        conn = sqlite3.connect('CommodityDatabase.db')     # create a new DataBase with the name "CommodityDatabase.db"
    except:
        print('Could not Create Database.') 
else:
    try:
        conn = sqlite3.connect('CommodityDatabase.db')     # if Database doesn't exist, Create a new DataBase with the name "CommodityDatabase.db"
    except:
        print('Could not Create Database.')                # create a new DataBase with the name "CommodityDatabase.db"
        
cur = conn.cursor()                                        # creating a object with Cursor class so that it can invoke methods which execute SQLite statements
cur.execute("CREATE TABLE CommodityTable (Ticker TEXT, Price REAL)") # creating table with the column name Ticker and Price, Ticker datatype is TEXT and Price datatype is REAL


################### Web Scraping ###################
result = requests.get('https://finance.yahoo.com/commodities') # calling the website's url using requests library and saving in a variable

def price_of_commodities(name_of_commodity):               # function that will parse the commodity price from saved website's data, commodity name will be sent as parameter
    soup = bs4.BeautifulSoup(result.text, 'lxml')          # parsing the saved raw website data using Beautiful Soup library
    list1 = []                                             # declaring a empty list: List1
    keyword = '>'+name_of_commodity                        # preparing keyword that will be searched as String in each row of the table ex: >Crude Oil, >Silver, >Gold
    for element in soup.select('tr', {'class':'simpTblRow'}): # iterating each and every table row "tr" which have "class: simpTblRow", used Beautiful Soup Select method 
        if keyword in str(element):                        # converting each row element into string and checking if the keyword exists or not
            list1.append(element)                          # if keyword is found in any row then that row will be inserted into the List1
    list2 = list1[0].select('td')                          # each table row 'tr' has multiple data 'td', so selecting all the 'td' and inserting into another list named List2
    return list2[2].text                                   # as per website, the third 'td' is the Price of the Commodity, here returning the third data as price


################### Fetching Price, Writing in File, Populating in Database ###################
commodities = ['Crude Oil','Gold','Silver']                # List of all commodities which will be searched

for commodity in commodities:                              # iterating through the commodity list
    with open('commodity_prices.txt','a') as fp:           # opening the "commodity_prices.txt"
        fp.write(commodity + ': ' + price_of_commodities(commodity) + '\n') # Writing the Commodity Name and Commodity Price in the file; getting the commodity price using the price_of_commodities function
    cur.execute("INSERT INTO CommodityTable (Ticker, Price) VALUES (?,?)", (commodity,price_of_commodities(commodity))) # inserting the commodity name in as Ticker and commodity price as Price; getting the commodity price using the price_of_commodities function

conn.commit()                                              # committing all the changes of Database
cur.close()                                                # closing the Cursor
conn.close()                                               # closing the Database connection

In [8]:
# this portion is for checking the database
import pandas as pd

conn = sqlite3.connect('CommodityDatabase.db')
df_emp = pd.read_sql_query("SELECT * FROM CommodityTable", conn)
df_emp

,Ticker,Price
0,Crude Oil,104.24
1,Gold,"1,892.10"
2,Silver,22.71


In [9]:
# run this to close the connection, otherwise next run will raise error
conn.close()